In [1]:
# parsing arguments for batch execution
import argparse
# JSON persistence
import json
# directory-read operations
import os

In [2]:
# manage directory-read operations using Posix paths
from pathlib import Path

In [3]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [4]:
# just a temporary workaround
# pass this code to the setup.py file of the final module
import sys
_ROOT_DIR = '{0}/gcp/cbidmltsf'.format(os.getenv("HOME"))
sys.path.append(_ROOT_DIR)

In [5]:
from dplstm.data import make_input_fn
from dplstm.model import DPLSTM

In [6]:
# build parameters dictionary for interactive execution
# in batch execution, this dictionary comes from parsed cli arguments

parameters = {
    'model_dir': 'lstm_60',
    'learning_rate': 0.01,
    'num_epochs': 20,  # NOT REQUIRED IN DISTRIBUTED MODE, IT IS OVERRIDDEN BY MAX_TRAIN_STEPS
    'max_train_steps': 20000,
    'train_batch_size': 32,
    'eval_interval': 300,
    'keep_checkpoint_max': 3,
    'eval_steps': None,
    'eval_batch_size': 2**16,
    'start_delay_secs': 600,
    'throttle_secs': 600,
    'train_data_path': '{0}/data/tfrecord/train.tfrecord'.format(_ROOT_DIR),
    'eval_data_path': '{0}/data/tfrecord/val.tfrecord'.format(_ROOT_DIR),
    'test_data_path': '{0}/data/tfrecord/test.tfrecord'.format(_ROOT_DIR),    
}

In [7]:
# support for log files
import logging

# advanced numerical operations
import numpy as np

import tensorflow as tf

_LOG_PATH = '{0}/logs'.format(_ROOT_DIR)
logging.basicConfig(filename='{}/test.log'.format(_LOG_PATH),
                    level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(threadName)s -  %(levelname)s - %(message)s')

logging.info('Logging to {}/test.log.'.format(_LOG_PATH))

# ToDo: pass a complex dictionary as a starting point for the LSTM network chromosome
'''
    Deep/Parallel LSTM network chromosome includes:
    m_hour: [24, 12, 8, etc], tau=1, implement this variable later and work now with m_hour=24
    m_day: [7, 14, etc], tau=7, implement this variable later and work now with m_day=7
    m_week: [4, 8, 12, etc], tau=168, implement this variable later and work now with m_week=4
    hidden_hour: number of hidden units at hour-resolution
    hidden_day: number of hidden units at day-resolution
    hidden_week: number of hidden units at week-resolution
    levels_hour: number of levels in stacked LSTM for prediction at hour-resolution
    levels_day: number of levels in stacked LSTM for prediction at day-resolution
    levels_week: number of levels in stacked LSTM for prediction at week-resolution
'''

_LOG_PATH = '{0}/logs'.format(_ROOT_DIR)
logging.basicConfig(filename='{}/test.log'.format(_LOG_PATH),
                    level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(threadName)s -  %(levelname)s - %(message)s')

logging.info('Logging to {}/test.log.'.format(_LOG_PATH))

In [8]:
# start notebook from batch script, but first
# remove main function and transfer everything to first-level code cells

In [9]:
tf.logging.set_verbosity(tf.logging.INFO)

In [10]:
def time_series_forecaster(features, labels, mode):
    '''
    this is the custom estimator
    '''
    # instantiate network topology from the corresponding class
    forecaster_topology = DPLSTM()

    # ToDo: global_step variable might be moved to TRAIN scope
    global_step = tf.train.get_global_step()

    # call operator to forecaster_topology, over features
    forecast = forecaster_topology(features)

    # predictions are stored in a dictionary for further use at inference stage
    predictions = {
        "forecast": forecast
    }

    # CHANGE MODEL FUNCTION STRUCTURE ACCORDING TO GILLARD'S ARCHITECTURE

    # Estimator in TRAIN or EVAL mode
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        # use labels and predictions to define training loss and evaluation loss
        # generate summaries for TensorBoard
        with tf.name_scope('loss'):
            mean_squared_error = tf.losses.mean_squared_error(
                labels=labels, predictions=forecast, scope='loss')
            tf.summary.scalar('loss', mean_squared_error)

        with tf.name_scope('val_loss'):
            val_loss = tf.metrics.mean_squared_error(
                labels=labels, predictions=forecast, name='mse')
            tf.summary.scalar('val_loss', val_loss[1])

        # this is only required for PREDICT mode
        prediction_hooks = None

        # Estimator in TRAIN mode ONLY
        if mode == tf.estimator.ModeKeys.TRAIN:
            # This is needed for batch normalization, but has no effect otherwise
            update_ops = tf.get_collection(key=tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(control_inputs=update_ops):
                train_op = tf.contrib.layers.optimize_loss(
                    loss=mean_squared_error,
                    global_step=global_step,
                    learning_rate=parameters['learning_rate'],
                    optimizer="Adam")
            # Create a hook to print acc, loss & global step every 100 iter
            train_hook_list = []
            train_tensors_log = {'val_loss': val_loss[1],
                                 'loss': mean_squared_error,
                                 'global_step': global_step}
            train_hook_list.append(tf.train.LoggingTensorHook(
                tensors=train_tensors_log, every_n_iter=100))

            predictions = None  # this is not required in TRAIN mode
            loss = mean_squared_error
            eval_metric_ops = None
            training_hooks = train_hook_list
            evaluation_hooks = None

        else:  # Estimator in EVAL mode ONLY
            loss = mean_squared_error
            train_op = None
            training_hooks = None
            eval_metric_ops = {'val_loss': val_loss}
            evaluation_hooks = None

    # Estimator in PREDICT mode ONLY
    else:
        loss = None
        train_op = None
        eval_metric_ops = None
        training_hooks = None
        evaluation_hooks = None
        prediction_hooks = None  # this might change as we are in PREDICT mode

    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops,
        # export_outputs=not_used_yet (for TensorFlow Serving, redirected from predictions if omitted)
        # training_chief_hooks=not_used_yet
        # ToDo: verify use of training_hooks
        # temporarily disable training hooks
        # training_hooks=training_hooks,
        training_hooks=training_hooks,
        # scaffold=not_used_yet
        evaluation_hooks=evaluation_hooks,
        prediction_hooks=prediction_hooks
    )

# ToDo: evaluate the convenience of packaging execution in a tf.app
# in the meantime, run the estimator outside tf.app package
# tf.logging.set_verbosity(tf.logging.INFO)

In [11]:
# ensure file writer cache is clear for TensorBoard events file
# ToDo: verify if this operation is required
tf.summary.FileWriterCache.clear()

In [12]:
# parameters required for RunConfig()
# _EVAL_INTERVAL = 300  # how often checkpoints are written out, given in seconds
# _KEEP_CHECKPOINT_MAX = 3  # how many checkpoints to keep
# _MAX_TRAIN_STEPS = 16000
# _TRAIN_BATCH_SIZE = 2**5
# _EVAL_STEPS = None  # if None, evaluate on entire dataset
# _EVAL_BATCH_SIZE = 2**16
# _START_DELAY_SECONDS = 600
# _THROTTLE_SECONDS = 600

In [13]:
# instantiate base estimator class for custom model function
tsf_estimator = tf.estimator.Estimator(
    model_fn=time_series_forecaster,
    # no parameters passed at this point
    # params=hparams,
    # parameters passed in original model include: train_data_path, batch_size, augment, train_steps
    config=tf.estimator.RunConfig(
        save_checkpoints_secs=parameters['eval_interval'],
        keep_checkpoint_max=parameters['keep_checkpoint_max']
    ),
    model_dir='{0}/dplstm/{1}'.format(_ROOT_DIR, parameters['model_dir']))

INFO:tensorflow:Using config: {'_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb73ae91d68>, '_log_step_count_steps': 100, '_is_chief': True, '_save_summary_steps': 100, '_service': None, '_train_distribute': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_save_checkpoints_secs': 300, '_master': '', '_protocol': None, '_experimental_max_worker_delay_secs': None, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_device_fn': None, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_eval_distribute': None, '_evaluation_master': '', '_task_id': 0, '_keep_checkpoint_max': 3, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_model_dir': '/home/jupyter/gcp/cbidmltsf/dplstm/lstm_50', '_global_id_in_cluster': 0, '_session_creation_timeout_secs': 7200}


In [14]:
# set estimator's train_spec to use train_input_fn and train it for many, many steps
train_spec = tf.estimator.TrainSpec(
    input_fn=make_input_fn(
        tfrecord_path=parameters['train_data_path'],
        batch_size=parameters['train_batch_size'],
        mode=tf.estimator.ModeKeys.TRAIN
    ),
    max_steps=parameters['max_train_steps'])

In [15]:
# ready to create a new serving input function

In [16]:
# features dictionary for parsing TFrecord files
read_features = {
    'hourly': tf.io.VarLenFeature(dtype=tf.float32),
    'daily': tf.io.VarLenFeature(dtype=tf.float32),
    'weekly': tf.io.VarLenFeature(dtype=tf.float32),
    'target': tf.io.FixedLenFeature([], dtype=tf.float32)}

In [17]:
def _parse_dataset_function(example_proto, objective_shape):
    # parse the input tf.Example proto using the dictionary called read_features (above defined)
    row = tf.io.parse_single_example(example_proto, read_features)
    # pass objective shape as a list of lists [hourly_shape, daily_shape, weekly_shape]
    hourly = tf.reshape(row['hourly'].values, objective_shape[0])
    daily = tf.reshape(row['daily'].values, objective_shape[1])
    weekly = tf.reshape(row['weekly'].values, objective_shape[2])
    target = tf.reshape(row['target'], [1, ])
    return {'hourly': hourly, 'daily': daily, 'weekly': weekly}, target

In [18]:
def serving_input_fn():
    # it handles only one example at a time
    # TPU are not optimized for serving, so it is assumed the predictions server is CPU or GPU-based
    # inputs is equivalent to example protocol buffers
    feature_placeholders = {'example_bytes': tf.placeholder(tf.string, shape=())}
    _SERVING_OBJECTIVE_SHAPES = [[1, 24, 1], [1, 7, 1], [1, 4, 1]]
    # the serving input function does not require the label
    features, _ = _parse_dataset_function(feature_placeholders['example_bytes'], _SERVING_OBJECTIVE_SHAPES)
    # re-shape to original model spec
    
    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [19]:
# create exporter that uses serving_input_fn to create saved_model for serving
exporter = tf.estimator.LatestExporter(
    name = "exporter", 
    serving_input_receiver_fn = serving_input_fn)

In [20]:
# set estimator's eval_spec to use eval_input_fn and export saved_model
eval_spec = tf.estimator.EvalSpec(
    input_fn=make_input_fn(
        tfrecord_path=parameters['eval_data_path'],
        batch_size=parameters['eval_batch_size'],
        mode=tf.estimator.ModeKeys.EVAL
    ),
    steps=parameters['eval_steps'],  # use None to evaluate on the entire dataset
    exporters=exporter,
    start_delay_secs=parameters['start_delay_secs'],  # delay first evaluation
    throttle_secs=parameters['throttle_secs']  # evaluate at a different rate (usually longer) than checkpoint
)

In [21]:
# run train_and_evaluate loop
tf.estimator.train_and_evaluate(
    estimator=tsf_estimator,
    train_spec=train_spec,
    eval_spec=eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be inc

({'global_step': 20000, 'loss': 0.007421616, 'val_loss': 0.007421616},
 [b'/home/jupyter/gcp/cbidmltsf/dplstm/lstm_50/export/exporter/1578931227'])

In [22]:
# get the directory of the latest saved model
_SAVED_MODEL_DIR = '{0}/dplstm/{1}/export/exporter'.format(_ROOT_DIR, parameters['model_dir'])

In [23]:
subdirs = [x for x in Path(_SAVED_MODEL_DIR).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
_LATEST_SAVED_MODEL_DIR = str(sorted(subdirs)[-1])

In [24]:
_LATEST_SAVED_MODEL_DIR

'/home/jupyter/gcp/cbidmltsf/dplstm/lstm_50/export/exporter/1578931227'

In [25]:
# build a prediction function
predict_fn = tf.contrib.predictor.from_saved_model(_LATEST_SAVED_MODEL_DIR)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /home/jupyter/gcp/cbidmltsf/dplstm/lstm_50/export/exporter/1578931227/variables/variables


In [26]:
# TODO: ORGANIZE ALL SHAPES TO AVOID UNNECESSARY RE-SHAPING OPERATIONS!!!

In [27]:
# now locate TFRecord test dataset and load it, need to parse the examples to a dictionary
test_dataset_filename = '{0}/data/tfrecord/test.tfrecord'.format(_ROOT_DIR)
test_dataset_filename

'/home/jupyter/gcp/cbidmltsf/data/tfrecord/test.tfrecord'

In [28]:
# now read the dataset from TFRecord file using non-deprecated methods from tf.data module
test_raw_dataset = tf.data.TFRecordDataset(test_dataset_filename)
test_raw_dataset 

<TFRecordDatasetV1 shapes: (), types: tf.string>

In [29]:
# disable the interactive session as it is not longer required

In [30]:
# sess = tf.InteractiveSession()

In [31]:
# can I access to the binary, string-based, raw dataset using a one-shot iterator?
# iterator = test_raw_dataset.make_one_shot_iterator()
# next_element = iterator.get_next()

# there is only one row in the raw dataset
# single_example = sess.run(next_element)

In [32]:
# single_example

In [33]:
# single_prediction = predict_fn({'example_bytes': single_example})

In [34]:
# single_prediction

In [35]:
# single_prediction['forecast'][0][0]

In [36]:
# finally, how to serve the saved model over the complete test dataset?
# must review software projects again!

In [37]:
dataset = test_raw_dataset.map(lambda row: row)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

In [38]:
# refine this iterator
# it is extremely slow!!!
# load to memory?
predictions_list = []
with tf.Session() as sess:
    try:
        while True:
            example = sess.run(next_element)
            predictions_list.append(predict_fn({'example_bytes': example}))
    except:
        pass

In [39]:
# why is the previous code cell so slow?
# it must be a better way to create the predictions list once the trained model is saved!

In [40]:
predictions_list[:10]

[{'forecast': array([[0.5199028]], dtype=float32)},
 {'forecast': array([[0.5460328]], dtype=float32)},
 {'forecast': array([[0.56626]], dtype=float32)},
 {'forecast': array([[0.56715137]], dtype=float32)},
 {'forecast': array([[0.5959195]], dtype=float32)},
 {'forecast': array([[0.60846704]], dtype=float32)},
 {'forecast': array([[0.61072195]], dtype=float32)},
 {'forecast': array([[0.60524976]], dtype=float32)},
 {'forecast': array([[0.59112555]], dtype=float32)},
 {'forecast': array([[0.55812615]], dtype=float32)}]

In [41]:
# predict based on saved model and plot results with Bokeh

In [42]:
# Anaconda Interactive Visualization
from bokeh.plotting import figure
from bokeh.plotting import output_file, save

In [43]:
# persistence for the scaler located in $DATA/scalers
from sklearn.externals import joblib

# reload scaler fitted model here
scaler = joblib.load('{0}/data/scalers/ci_LSTM_scaler.save'.format(_ROOT_DIR))

/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.19.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [44]:
len(predictions_list)

566

In [45]:
# move to array and re-scale
predictions = [p['forecast'][0][0] for p in predictions_list]
predictions = np.asarray(predictions)
pred_ci = scaler.inverse_transform(predictions.reshape(-1, 1))
pred_ci = np.squeeze(pred_ci)

In [46]:
pred_ci[:10]

array([4.2805333, 4.49567  , 4.662207 , 4.669546 , 4.906404 , 5.009712 ,
       5.0282774, 4.983223 , 4.866934 , 4.5952387], dtype=float32)

In [47]:
# ToDo: get ytarget_test array from test.tfrecord dataset to perform the following operation
# temporarily get the array from disk
y_test = np.load('{0}/data/arrays/y_test.npy'.format(_ROOT_DIR))
n = 0  # first step ahead
ytarget_test = y_test[:, n]
ytarget_test = ytarget_test.reshape(ytarget_test.shape[0], 1)

actual_ci = scaler.inverse_transform(ytarget_test)
actual_ci = np.squeeze(actual_ci)

In [48]:
_EQUIPMENT = 'CPE04105'

In [49]:
ci_predictions_fig = figure(title='Predicted Current Imbalance for ' + _EQUIPMENT,
                            background_fill_color='#E8DDCB',
                            plot_width=1800, plot_height=450, x_axis_type='datetime')

# ToDo: yts_test array is required to get timestamps for plot, then wire it now and get it from TFRecord later...
yts_test = np.load('{0}/data/arrays/yts_test.npy'.format(_ROOT_DIR), allow_pickle=True)

ci_predictions_fig.line(yts_test[:, n],
                        actual_ci, line_color='red',
                        line_width=1, alpha=0.7, legend='Actual')

ci_predictions_fig.line(yts_test[:, n],
                        pred_ci, line_color='blue',
                        line_width=1, alpha=0.7, legend='Predicted')

ci_predictions_fig.legend.location = "top_right"
ci_predictions_fig.legend.background_fill_color = "darkgrey"

ci_predictions_fig.xaxis.axis_label = 'Timestamp'
ci_predictions_fig.yaxis.axis_label = 'Current Imbalance [%]'

# output_file('{0}/plots/'.format(_ROOT_DIR) + '{:03d}'.format(n) + '.html', title=_EQUIPMENT)
output_file('{0}/plots/'.format(_ROOT_DIR) + '{}'.format(parameters['model_dir']) + '.html', title=_EQUIPMENT)
save(ci_predictions_fig)

'/home/jupyter/gcp/cbidmltsf/plots/lstm_50.html'